In [33]:
#Importation de librairies
import pandas as pd 
import numpy as np 
# Importer les bibliothèques nécessaires
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.neighbors import KNeighborsRegressor


In [30]:
data=pd.read_csv(r'..\data\data_clean\data_preprocess.csv')
data


,TYPE,PRICE,BEDS,BATH,PROPERTYSQFT,BOROUGH
0,Multi-family home for sale,1100000,6,3,2837,Bronx
1,Multi-family home for sale,925000,5,2,2750,Bronx
2,Multi-family home for sale,700000,6,2,3053,Bronx
3,Autre_Type,119000,3,2,2184,Bronx
4,Multi-family home for sale,600000,2,2,2184,Bronx
...,...,...,...,...,...,...
4791,Co-op for sale,280000,2,1,1050,Bronx
4792,Co-op for sale,375000,2,1,2184,Bronx
4793,Co-op for sale,167999,1,1,2184,Bronx
4794,Co-op for sale,309000,2,1,2184,Bronx


In [54]:
def prepare_features(data, categorical_columns, target_column):
    """
    Prépare les données en encodant les colonnes catégoriques et standardisant les colonnes numériques.
    """
    # Séparer les variables explicatives et la cible
    X = data.drop(columns=[target_column])
    y = data[target_column]
    
    # Encoder les colonnes catégoriques
    encoder = OneHotEncoder(sparse=False, drop='first')
    encoded_categorical = encoder.fit_transform(X[categorical_columns])
    encoded_columns = pd.DataFrame(encoded_categorical, columns=encoder.get_feature_names_out(categorical_columns))
    
    # Fusionner les colonnes encodées avec les autres colonnes
    X = pd.concat([X.drop(columns=categorical_columns), encoded_columns], axis=1)
    
    # Standardiser les colonnes numériques
    numerical_columns = ['BEDS', 'BATH', 'PROPERTYSQFT']
    scaler = StandardScaler()
    X[numerical_columns] = scaler.fit_transform(X[numerical_columns])
    
    return X, y


def split_data(X, y, test_size=0.2, random_state=42):
    """
    Divise les données en ensembles d'entraînement et de test.
    """
    return train_test_split(X, y, test_size=test_size, random_state=random_state)

def kfold_cross_validation(X, y, model, n_splits=5, random_state=42):
    """
    Effectue une validation croisée avec K-Fold sur un modèle donné.
    """
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    mse_scores = []
    r2_scores = []
    
    for train_index, test_index in kf.split(X):
        # Diviser les données selon les indices des folds
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        # Entraîner le modèle
        model.fit(X_train, y_train)
        # Faire des prédictions
        y_pred = model.predict(X_test)
        # Calculer les métriques pour ce fold
        mse_scores.append(mean_squared_error(y_test, y_pred))
        r2_scores.append(model.score(X_test, y_test))  # R² via la méthode `score`
    
    return np.mean(mse_scores), np.mean(r2_scores)

def train_random_forest(X_train, y_train, n_estimators=1000, random_state=42):
    """
    Entraîne un modèle Random Forest Regressor.
    """
    model = RandomForestRegressor(n_estimators=n_estimators, random_state=random_state)
    model.fit(X_train, y_train)
    return model

def train_knn_model(X_train, y_train, n_neighbors=5):
    """
    Entraîne un modèle KNN avec les données fournies.
    """
    knn_model = KNeighborsRegressor(n_neighbors=n_neighbors)
    knn_model.fit(X_train, y_train)
    return knn_model

def train_and_evaluate(data, categorical_columns, target_column):
    """
    Prépare les données, entraîne un modèle Random Forest et effectue une K-Fold Cross-Validation.
    """
    # Préparer les données
    X, y = prepare_features(data, categorical_columns, target_column)
    # Initialiser le modèle
    model = RandomForestRegressor(n_estimators=1000, random_state=42)
    # Effectuer la validation croisée
    mean_mse, mean_r2 = kfold_cross_validation(X, y, model)
    return mean_mse, mean_r2

def evaluate_model(model, X_test, y_test):
    """
    Évalue un modèle sur un ensemble de test.
    """
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return mse, r2


In [55]:
if __name__ == "__main__":
    # Définir les colonnes catégoriques et la cible
    categorical_columns = ['TYPE', 'BOROUGH']
    target_column = 'PRICE'
    
    # Préparer les caractéristiques
    X, y = prepare_features(data, categorical_columns, target_column)
    
    # Diviser les données en ensembles d'entraînement et de test
    X_train, X_test, y_train, y_test = split_data(X, y)
    
    # Entraîner le modèle Random Forest
    model = train_random_forest(X_train, y_train)
    
    # Évaluer le modèle
    mse, r2 = evaluate_model(model, X_test, y_test)
    print(f"Mean Squared Error (MSE): {mse:.2f}")
    print(f"Coefficient of Determination (R²): {r2:.2f}")


Mean Squared Error (MSE): 4722211458856.25
Coefficient of Determination (R²): 0.50


In [52]:
if __name__ == "__main__":
    
    # Définir les colonnes catégoriques et la cible
    categorical_columns = ['TYPE', 'BOROUGH']
    target_column = 'PRICE'
    
    # Entraîner et évaluer le modèle
    mean_mse, mean_r2 = train_and_evaluate(data, categorical_columns, target_column)
    
    print(f"Mean MSE (k-fold): {mean_mse:.2f}")
    print(f"Mean R² (k-fold): {mean_r2:.2f}")

Mean MSE (k-fold): 6508420969961.52
Mean R² (k-fold): 0.60


In [53]:
if __name__ == "__main__":
    # Définir les colonnes catégoriques et la cible
    categorical_columns = ['TYPE', 'BOROUGH']
    target_column = 'PRICE'
    
    # Préparer les caractéristiques
    X_scaled, y = prepare_features(data, categorical_columns, target_column)
    
    # Diviser les données en ensembles d'entraînement et de test
    X_train, X_test, y_train, y_test = split_data(X_scaled, y)
    
    # Entraîner le modèle KNN
    knn_model = train_knn_model(X_train, y_train, n_neighbors=15)
    
    # Évaluer le modèle
    mse, r2 = evaluate_model(knn_model, X_test, y_test)
    print(f"Mean Squared Error (MSE): {mse:.2f}")
    print(f"Coefficient of Determination (R²): {r2:.2f}")

Mean Squared Error (MSE): 4256319267448.76
Coefficient of Determination (R²): 0.55
